In [58]:
from deepmultilingualpunctuation import PunctuationModel
from datasets import load_dataset

import evaluate
import seqeval

import spacy
import json


In [57]:
nlp = spacy.load("en_core_web_sm")

In [29]:
seqeval = evaluate.load("seqeval")

In [30]:
model = PunctuationModel()

/Users/dtemnov/Projects/comma-placement/.venv/lib/python3.11/site-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.NONE"` instead.
  warnings.warn(


In [31]:
dataset_path = "just097/wiki-comma-placement"
wiki_comma_placement = load_dataset(dataset_path)
LABEL_LIST = ["O", "B-COMMA"]

In [ ]:
wiki_comma_placement["test"][21]

In [55]:
try_sample = wiki_comma_placement["test"][21]

text = " ".join(try_sample["tokens"]).strip()
clean_text = model.preprocess(text)
punct_text = model.restore_punctuation(text)
print(punct_text)

Kovacs only had two campus visits: Division II, Hillsdale and Toledo ( a school 13 miles from his high school ).


In [82]:
test_subset = wiki_comma_placement["test"]

In [62]:
def sentence_to_sample(sentence: str) -> str:
    sentence = sentence.strip()
    words = [word.text for word in nlp(sentence)]
    tags = []
    clean_words = []
    for i in range(len(words) - 1):
        if words[i] == ",":
            continue
        if words[i + 1] == ",":
            clean_words.append(words[i])
            tags.append(1)
        else:
            clean_words.append(words[i])
            tags.append(0)
    clean_words.append(words[-1])
    tags.append(0)
    assert len(tags) == len(clean_words)
    return {"tokens": clean_words, "tags": tags}

In [75]:
def restore_text(example):
    text = " ".join(example["tokens"]).strip()
    clean_text = model.preprocess(text)
    clean_text = " ".join(clean_text).strip()
    punct_text = model.restore_punctuation(clean_text)
    return sentence_to_sample(punct_text)

## Generate restored sentences with baseline.

In [ ]:
restored_texts = test_subset.map(restore_text)

In [52]:
def predicted_to_tags(labled_words):
    pred_tags = [1 if token[1] == ',' else 0 for token in labled_words]
    pred_tags.append(0)
    return pred_tags


In [ ]:
predicted_tags = predicted_to_tags(labled_words)

print(try_sample["tags"])
print(predicted_tags)

assert len(try_sample["tags"]) == len(predicted_tags)

In [38]:
def compute_metrics(predicted_tags, gt_tags):
    true_predictions = [[LABEL_LIST[p] for p in predicted_tags]]
    true_labels = [[LABEL_LIST[t] for t in gt_tags]]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [39]:
compute_metrics(predicted_tags, try_sample["tags"])

ValueError: Found input variables with inconsistent numbers of samples:
[19]
[18]